In [2]:
import pandas as pd

df = pd.read_csv('data_preprocessing/data/processed_reviews.csv')

df.head()

,stars,useful,funny,cool,lemmatized_text
0,3.0,0,0,0,decide eat aware going take 2 hour beginning e...
1,5.0,1,0,1,ive taken lot spin class year nothing compare ...
2,3.0,0,0,0,family diner buffet eclectic assortment large ...
3,5.0,1,0,1,wow yummy different delicious favorite lamb cu...
4,4.0,1,0,1,cute interior owner gave u tour upcoming patio...


In [3]:
df = df.dropna(axis=0)

if df['lemmatized_text'].isna().any():
    print("The DataFrame contains np.nan values.")
else:
    print("The DataFrame does not contain np.nan values.")


The DataFrame does not contain np.nan values.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Creating a document-term matrix
vectorizer = TfidfVectorizer()
doc_term_matrix = vectorizer.fit_transform(df['lemmatized_text'])

# Computing TF-IDF
tf_idf = pd.DataFrame(doc_term_matrix.toarray(), columns=vectorizer.get_feature_names())

# Saving TF-IDF matrix
tf_idf.to_csv('./data_preprocessing/data/tf_idf.csv', index=False)

print("Completed")